In [1]:
import os, sys
import numpy as np
import pandas as pd
import json

# from txyl_common.biocyc_facade.pgdb import Pgdb, Dat, Traceable
from local.caching import load, save

In [2]:
root = './cache/'
usage_data = []
for f in sorted(list(os.listdir(root))):
    if not f.startswith('metabolite_usage'): continue
    with open(f'{root}/{f}') as j:
        usage_data += json.load(j)
    print(f'{f} {" "*25}', end='\r')

len(usage_data)

19999

In [3]:
for id, ins, outs, cats in usage_data:
    print(ins)
    break

{'L-CYSTATHIONINE': 3, 'UTP': 8, 'CPD-12575': 3, 'PPI': 12, 'GLC-1-P': 4, 'PROTON': 333, 'ACP': 5, 'MALONYL-COA': 5, 'Hydroxymethylbilane-Synthase-ES2': 1, 'PORPHOBILINOGEN': 5, 'HYDROGEN-PEROXIDE': 13, 'MET': 12, 'WATER': 400, 'CPD0-2351': 2, 'NAD': 62, 'Dihydro-Lipoyl-Proteins': 1, 'CPD-13339': 2, 'HYDROXYMUCONATE-SALD': 1, 'NADH': 52, 'S-NITROSOGLUTATHIONE': 2, '2-D-THREO-HYDROXY-3-CARBOXY-ISOCAPROATE': 4, 'CPD-9451': 2, 'PROTEIN-L-BETA-ISOASPARTATES': 1, 'S-ADENOSYLMETHIONINE': 71, 'CPD-17968': 1, 'Protein-With-N-Terminal-X-Pro': 2, 'Purine-ribonucleosides-5-PPP': 1, 'ADENOSYLCOBINAMIDE': 3, 'ATP': 210, 'LysW-L-glutamate': 1, 'CPD-12185': 1, 'Alpha-lactose': 1, 'Acceptor': 8, 'BryP-ACP': 1, 'Hydroxymethylbilane-Synthase-ES3': 1, 'Pi': 31, 'CPD-16459': 1, 'MUTATED-TRNA': 1, 'Oxo-glutarate-dehydrogenase-DH-lipoyl': 1, 'DIHYDROXY-ACETONE-PHOSPHATE': 10, 'Arsenite-ArsD': 1, 'OXYGEN-MOLECULE': 42, 'Cytochromes-C-Reduced': 2, 'CPD-679': 1, 'GTP': 16, 'L-CITRULLINE': 3, 'CARBAMOYL-P': 4, 

In [4]:
fi, fo, fc = {}, {}, {}
for id, ins, outs, cats in usage_data:
    for ref, lst in zip([fi, fo, fc], [ins, outs, cats]):
        for c in lst:
            if c in ref: continue
            ref[c] = len(ref)

flens = len(fi), len(fo), len(fc)
flens

(14845, 16711, 1125)

In [5]:
features = {}
for pre, ref in zip("i, o, c".split(', '), [fi, fo, fc]):
    for k, _ in ref.items():
        features[f"{pre}_{k}"] = len(features)
len(features)

32681

In [6]:
xlen, ylen = len(usage_data), len(features)
ids = []
mat = np.ndarray(shape=(xlen, ylen), dtype=np.ushort)
for row_i, (id, ins, outs, cats) in enumerate(usage_data):
    ids.append(id)
    for pre, usage in zip("i, o, c".split(', '), [ins, outs, cats]):
        for k, c in usage.items():
            key = f"{pre}_{k}"
            i = features[key]
            mat[row_i, i] = c

mat

array([[3, 8, 3, ..., 0, 0, 0],
       [1, 8, 5, ..., 0, 0, 0],
       [1, 8, 5, ..., 0, 0, 0],
       ...,
       [3, 7, 5, ..., 0, 0, 0],
       [0, 7, 1, ..., 0, 0, 0],
       [2, 6, 3, ..., 0, 0, 0]], dtype=uint16)

In [11]:
flst = ['']*len(features)
for k, i in features.items():
    flst[i] = k

In [14]:
df = pd.DataFrame(mat, index=ids, columns=flst, dtype=np.ushort)

In [15]:
save('biocyc_metabolite_usage', df)

compressing & caching data to [{WORKSPACE}/main/prep/cache/biocyc_metabolite_usage.pkl.gz]


In [27]:
flst[-5:]

['c_UDP-GLUCURONATE',
 'c_CPD-12575',
 'c_NOREPINEPHRINE',
 'c_Nucleoside-Monophosphates',
 'c_3-4-DIHYDROXYBENZOATE']

In [6]:
# root = "../../data/txyl_local/biocyc/pgdbs"
# pfs = os.listdir(root)
# total = len(pfs)
# for i, pf in enumerate(pfs):
#     print(f"{i+1} of {total}: {pf} {' '*50}", end='\r')
#     db = Pgdb(f"{root}/{pf}")
    
#     def get_ins_outs(v: dict):
#         direction: str = v.get('REACTION-DIRECTION', ['unk'])[0]
#         lefts = set(v.get('LEFT', []))
#         rights = set(v.get('RIGHT', []))

#         left_o = lefts.difference(rights)
#         right_o = rights.difference(lefts)
#         catalysts = lefts.intersection(rights)
        
#         if 'LEFT' not in direction or 'RIGHT' not in direction: # reversible
#             all = lefts.union(rights)
#             return all, all, catalysts
#         else:
#             li = direction.index('LEFT')
#             ri = direction.index('RIGHT')

#             if li < ri:
#                 return left_o, right_o, catalysts
#             else:
#                 return right_o, left_o, catalysts

#     consumption, production, catalyst_use = {}, {}, {}
#     def addc(d: dict, v: str):
#         d[v] = d.get(v, 0)+1

#     for k, v in db.GetDataTable(Dat.REACTIONS).items():
#         ins, outs, cats = get_ins_outs(v)
#         for lst, ref in zip([ins, outs, cats], [consumption, production, catalyst_use]):
#             for mol in lst:
#                 addc(ref, mol)

#     return 


In [11]:
# def get_ins_outs(v: dict):
#     direction: str = v.get('REACTION-DIRECTION', ['unk'])[0]
#     lefts = set(v.get('LEFT', []))
#     rights = set(v.get('RIGHT', []))

#     left_o = lefts.difference(rights)
#     right_o = rights.difference(lefts)
#     catalysts = lefts.intersection(rights)
    
#     if 'LEFT' not in direction or 'RIGHT' not in direction: # reversible
#         all = lefts.union(rights)
#         return all, all, catalysts
#     else:
#         li = direction.index('LEFT')
#         ri = direction.index('RIGHT')

#         if li < ri:
#             return left_o, right_o, catalysts
#         else:
#             return right_o, left_o, catalysts

# consumption, production, catalyst_use = {}, {}, {}
# def addc(d: dict, v: str):
#     d[v] = d.get(v, 0)+1

# for k, v in db.GetDataTable(Dat.REACTIONS).items():
#     ins, outs, cats = get_ins_outs(v)
#     for lst, ref in zip([ins, outs, cats], [consumption, production, catalyst_use]):
#         for mol in lst:
#             addc(ref, mol)

In [4]:
# assign_methods = set()

# for k, v in db.GetDataTable(Dat.ENZYMES).items():
#     ak = "BASIS-FOR-ASSIGNMENT"
#     assert ak in v
#     assign_methods.add(v[ak][0])
#     print(v)
#     break
    
# assign_methods

{'TYPES': ['Enzymatic-Reactions'], 'COMMON-NAME': ['acetyl-CoA carboxylase carboxyltransferase subunit alpha/beta'], 'BASIS-FOR-ASSIGNMENT': [':AUTOMATED-NAME-MATCH'], 'ENZYME': ['DZ96_RS0232580-MONOMER'], 'INSTANCE-NAME-TEMPLATE': ['ENZRXN-*'], 'PHYSIOLOGICALLY-RELEVANT?': ['T'], 'REACTION': ['ACETYL-COA-CARBOXYLTRANSFER-RXN']}


{':AUTOMATED-NAME-MATCH'}

In [5]:
# BL = {':INFERRED-TRANSPORT-RXN', ':PATHWAY-HOLE-FILLER'}